In [ ]:
 import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from typing import List, Tuple, Optional
import matplotlib.patches as patches


def validate_inputs(A: np.ndarray, b: np.ndarray, x0: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Valida y normaliza las entradas del sistema."""
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float).reshape(-1, 1)
    x0 = np.array(x0, dtype=float).reshape(-1, 1)

    if A.shape[0] != A.shape[1]:
        raise ValueError("La matriz A debe ser cuadrada.")
    if b.shape[0] != A.shape[0]:
        raise ValueError("Dimensiones incompatibles entre A y b.")
    if x0.shape[0] != A.shape[0]:
        raise ValueError("Dimensiones incompatibles entre A y x0.")
    
    # Verificar que la diagonal no tenga ceros
    if np.any(np.diag(A) == 0):
        raise ValueError("La matriz A no puede tener ceros en la diagonal.")

    return A, b, x0


def check_convergence(A: np.ndarray) -> Tuple[bool, bool]:
    """Verifica las condiciones de convergencia para ambos métodos."""
    # Diagonal dominante estricta
    diagonal_dominant = True
    for i in range(A.shape[0]):
        diagonal_sum = sum(abs(A[i, j]) for j in range(A.shape[1]) if j != i)
        if abs(A[i, i]) <= diagonal_sum:
            diagonal_dominant = False
            break
    
    # Matriz de iteración de Jacobi
    D = np.diag(np.diag(A))
    L = np.tril(A, -1)
    U = np.triu(A, 1)
    
    # Radio espectral de Jacobi
    D_inv = np.linalg.inv(D)
    T_jacobi = -D_inv @ (L + U)
    jacobi_converges = np.max(np.abs(np.linalg.eigvals(T_jacobi))) < 1
    
    # Radio espectral de Gauss-Seidel
    T_gs = -np.linalg.inv(D + L) @ U
    gs_converges = np.max(np.abs(np.linalg.eigvals(T_gs))) < 1
    
    return jacobi_converges, gs_converges


def gauss_jacobi(A: np.ndarray, b: np.ndarray, x0: np.ndarray, 
                 tol=1e-6, max_iter=100) -> Tuple[np.ndarray, List[np.ndarray], List[float]]:
    """Método de Gauss-Jacobi mejorado con tracking de errores."""
    A, b, x0 = validate_inputs(A, b, x0)
    n = A.shape[0]
    x = x0.copy()
    trajectory = [x.copy()]
    errors = [np.inf]

    for iteration in range(max_iter):
        x_new = np.zeros_like(x)
        for i in range(n):
            # Suma de términos no diagonales usando x anterior
            s = sum(A[i, j] * x[j] for j in range(n) if j != i)
            x_new[i] = (b[i] - s) / A[i, i]

        trajectory.append(x_new.copy())
        error = np.linalg.norm(x_new - x, ord=np.inf)
        errors.append(error)
        
        if error < tol:
            print(f"Jacobi convergió en {iteration + 1} iteraciones")
            break
        x = x_new
    else:
        print(f"Jacobi alcanzó el máximo de iteraciones ({max_iter})")

    return x, trajectory, errors


def gauss_seidel(A: np.ndarray, b: np.ndarray, x0: np.ndarray, 
                 tol=1e-6, max_iter=100) -> Tuple[np.ndarray, List[np.ndarray], List[float]]:
    """Método de Gauss-Seidel mejorado con tracking de errores."""
    A, b, x0 = validate_inputs(A, b, x0)
    n = A.shape[0]
    x = x0.copy()
    trajectory = [x.copy()]
    errors = [np.inf]

    for iteration in range(max_iter):
        x_prev = x.copy()
        for i in range(n):
            # Suma usando valores actualizados (j < i) y anteriores (j > i)
            s = sum(A[i, j] * x[j] for j in range(i)) + sum(A[i, j] * x_prev[j] for j in range(i+1, n))
            x[i] = (b[i] - s) / A[i, i]

        trajectory.append(x.copy())
        error = np.linalg.norm(x - x_prev, ord=np.inf)
        errors.append(error)
        
        if error < tol:
            print(f"Gauss-Seidel convergió en {iteration + 1} iteraciones")
            break
    else:
        print(f"Gauss-Seidel alcanzó el máximo de iteraciones ({max_iter})")

    return x, trajectory, errors


def plot_static_comparison(traj_jacobi: List[np.ndarray], 
                          traj_seidel: List[np.ndarray],
                          exact_sol: np.ndarray,
                          A: np.ndarray,
                          b: np.ndarray,
                          num_iters: int = 5):
    """Gráfico estático comparativo mejorado."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
    
    # Rango para graficar las líneas
    if A.shape[0] == 2:  # Sistema 2x2
        x_range = np.linspace(-1, 7, 100)
        
        # Ecuaciones del sistema: A[0,0]*x + A[0,1]*y = b[0]
        # y = (b[0] - A[0,0]*x) / A[0,1]
        y1 = (b[0] - A[0, 0] * x_range) / A[0, 1] if A[0, 1] != 0 else np.full_like(x_range, b[0]/A[0, 0])
        y2 = (b[1] - A[1, 0] * x_range) / A[1, 1] if A[1, 1] != 0 else np.full_like(x_range, b[1]/A[1, 0])
        
        # Graficar en ambos subplots
        for ax, method_name in zip([ax1, ax2], ['Jacobi', 'Gauss-Seidel']):
            ax.plot(x_range, y1, 'k--', linewidth=2, label=f'{A[0,0]}x + {A[0,1]}y = {b[0]}')
            ax.plot(x_range, y2, 'k:', linewidth=2, label=f'{A[1,0]}x + {A[1,1]}y = {b[1]}')
    
    # Procesar trayectorias
    trajectories = [traj_jacobi, traj_seidel]
    colors = ['blue', 'red']
    markers = ['o', 's']
    
    for i, (traj, ax, color, marker, method) in enumerate(zip(trajectories, [ax1, ax2], colors, markers, ['Jacobi', 'Gauss-Seidel'])):
        # Convertir trayectoria limitada
        traj_limited = traj[:min(num_iters+1, len(traj))]
        traj_array = np.array([p.flatten() for p in traj_limited])
        
        if len(traj_array) > 1:
            x_vals, y_vals = traj_array[:, 0], traj_array[:, 1]
            
            # Trayectoria completa
            ax.plot(x_vals, y_vals, '-', color=color, linewidth=2, alpha=0.7, 
                   label=f'Trayectoria {method}')
            
            # Puntos individuales
            ax.plot(x_vals[0], y_vals[0], marker='D', color='green', markersize=10, 
                   label='Punto inicial', zorder=5)
            
            if len(x_vals) > 2:
                ax.plot(x_vals[1:-1], y_vals[1:-1], marker=marker, color=color, 
                       markersize=8, label='Iteraciones', zorder=4)
            
            ax.plot(x_vals[-1], y_vals[-1], marker='*', color='orange', markersize=12, 
                   label=f'Iteración {len(x_vals)-1}', zorder=5)
        
        # Solución exacta
        ax.plot(exact_sol[0], exact_sol[1], 'X', color='red', markersize=12, 
               markeredgewidth=3, label='Solución exacta', zorder=6)
        
        ax.set_title(f'Método de {method} - {min(num_iters, len(traj)-1)} iteraciones')
        ax.set_xlabel('x₁')
        ax.set_ylabel('x₂')
        ax.grid(True, alpha=0.3)
        ax.legend()
        ax.set_xlim(-1, 7)
        ax.set_ylim(-1, 7)
    
    plt.tight_layout()
    plt.show()


def animate_method(trajectory: List[np.ndarray], 
                  exact_sol: np.ndarray,
                  A: np.ndarray,
                  b: np.ndarray,
                  method_name: str,
                  interval: int = 1000) -> FuncAnimation:
    """Crea animación para un método iterativo."""
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Configurar el gráfico base
    if A.shape[0] == 2:
        x_range = np.linspace(-1, 7, 100)
        y1 = (b[0] - A[0, 0] * x_range) / A[0, 1] if A[0, 1] != 0 else np.full_like(x_range, b[0]/A[0, 0])
        y2 = (b[1] - A[1, 0] * x_range) / A[1, 1] if A[1, 1] != 0 else np.full_like(x_range, b[1]/A[1, 0])
        
        ax.plot(x_range, y1, 'k--', linewidth=2, label=f'{A[0,0]}x + {A[0,1]}y = {b[0]}')
        ax.plot(x_range, y2, 'k:', linewidth=2, label=f'{A[1,0]}x + {A[1,1]}y = {b[1]}')
    
    # Solución exacta
    ax.plot(exact_sol[0], exact_sol[1], 'X', color='red', markersize=12, 
           markeredgewidth=3, label='Solución exacta', zorder=6)
    
    # Elementos animados
    traj_line, = ax.plot([], [], '-', color='blue', linewidth=2, alpha=0.7)
    points, = ax.plot([], [], 'o', color='blue', markersize=8)
    current_point, = ax.plot([], [], 'o', color='orange', markersize=12)
    
    ax.set_xlim(-1, 7)
    ax.set_ylim(-1, 7)
    ax.set_xlabel('x₁')
    ax.set_ylabel('x₂')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # Preparar datos de la trayectoria
    traj_array = np.array([p.flatten() for p in trajectory])
    
    def animate(frame):
        if frame < len(traj_array):
            # Trayectoria hasta el frame actual
            x_vals = traj_array[:frame+1, 0]
            y_vals = traj_array[:frame+1, 1]
            
            traj_line.set_data(x_vals, y_vals)
            points.set_data(x_vals[:-1], y_vals[:-1])
            current_point.set_data([x_vals[-1]], [y_vals[-1]])
            
            ax.set_title(f'Método de {method_name} - Iteración {frame}')
        
        return traj_line, points, current_point
    
    anim = FuncAnimation(fig, animate, frames=len(traj_array), 
                        interval=interval, blit=True, repeat=True)
    
    plt.show()
    return anim


def plot_convergence_analysis(errors_jacobi: List[float], 
                             errors_seidel: List[float]):
    """Analiza y grafica la convergencia de ambos métodos."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Gráfico de errores
    iterations_j = range(len(errors_jacobi))
    iterations_s = range(len(errors_seidel))
    
    ax1.semilogy(iterations_j, errors_jacobi, 'b-o', label='Jacobi', linewidth=2)
    ax1.semilogy(iterations_s, errors_seidel, 'r-s', label='Gauss-Seidel', linewidth=2)
    ax1.set_xlabel('Iteración')
    ax1.set_ylabel('Error (escala log)')
    ax1.set_title('Convergencia de los métodos')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Gráfico de razón de convergencia
    if len(errors_jacobi) > 2:
        ratios_j = [errors_jacobi[i+1]/errors_jacobi[i] for i in range(1, len(errors_jacobi)-1)]
        ax2.plot(range(1, len(ratios_j)+1), ratios_j, 'b-o', label='Jacobi', linewidth=2)
    
    if len(errors_seidel) > 2:
        ratios_s = [errors_seidel[i+1]/errors_seidel[i] for i in range(1, len(errors_seidel)-1)]
        ax2.plot(range(1, len(ratios_s)+1), ratios_s, 'r-s', label='Gauss-Seidel', linewidth=2)
    
    ax2.set_xlabel('Iteración')
    ax2.set_ylabel('Razón de convergencia')
    ax2.set_title('Razón de convergencia')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()


def solve_system_analysis(A: np.ndarray, b: np.ndarray, x0: np.ndarray, 
                         animate: bool = False, max_iter: int = 50):
    """Análisis completo del sistema con ambos métodos."""
    print("=== ANÁLISIS DEL SISTEMA ===")
    print(f"Matriz A:\n{A}")
    print(f"Vector b: {b.flatten()}")
    print(f"Punto inicial: {x0.flatten()}")
    
    # Verificar convergencia
    jacobi_conv, gs_conv = check_convergence(A)
    print(f"\nCondiciones de convergencia:")
    print(f"Jacobi converge: {jacobi_conv}")
    print(f"Gauss-Seidel converge: {gs_conv}")
    
    # Solución exacta
    exact_sol = np.linalg.solve(A, b)
    print(f"Solución exacta: {exact_sol.flatten()}")
    
    # Resolver con ambos métodos
    x_jacobi, traj_jacobi, errors_jacobi = gauss_jacobi(A, b, x0, max_iter=max_iter)
    x_seidel, traj_seidel, errors_seidel = gauss_seidel(A, b, x0, max_iter=max_iter)
    
    print(f"\nResultados:")
    print(f"Jacobi: {x_jacobi.flatten()} (error: {np.linalg.norm(x_jacobi - exact_sol):.2e})")
    print(f"Gauss-Seidel: {x_seidel.flatten()} (error: {np.linalg.norm(x_seidel - exact_sol):.2e})")
    
    # Visualizaciones
    plot_static_comparison(traj_jacobi, traj_seidel, exact_sol, A, b)
    plot_convergence_analysis(errors_jacobi, errors_seidel)
    
    if animate and A.shape[0] == 2:
        print("\nCreando animaciones...")
        anim_jacobi = animate_method(traj_jacobi, exact_sol, A, b, "Jacobi")
        anim_seidel = animate_method(traj_seidel, exact_sol, A, b, "Gauss-Seidel")
        return anim_jacobi, anim_seidel
    
    return None, None


if __name__ == "__main__":
    # Sistema de ejemplo
    A = np.array([[4, 1],
                  [1, 3]], dtype=float)
    b = np.array([7, 8], dtype=float)
    x0 = np.array([1, 1], dtype=float)
    
    # Análisis completo
    solve_system_analysis(A, b, x0, animate=True)
    
    # Ejemplo con el sistema original (modificado para convergencia)
    print("\n" + "="*50)
    print("SISTEMA ORIGINAL (MODIFICADO)")
    A2 = np.array([[3, 1],
                   [-1, 2]], dtype=float)
    b2 = np.array([6, 0], dtype=float)
    x0_2 = np.array([1, 1], dtype=float)
    
    solve_system_analysis(A2, b2, x0_2, animate=True)